In [2]:
from operator import add
import re
from collections import OrderedDict
from operator import itemgetter 
import itertools
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.119:7077") \
        .appName("TianruZ_common_crawl")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

In [3]:

# (*/*) - out of memory›
# ~6.4mins for 39496 files. (...00000/)  (takes 1 minute with 40 partitions)
# ~5 secs for 10 files (...00000/0*) 
# ~20 secs for 11110 files (...00000/1*) 


rdd = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.119:9000/crawl-wet/CC-MAIN-20200328074047-20200328104047-00000.warc.wet',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': 'WARC/1.0'}
)\
.cache() # Keep this RDD in memory!

rdd.count()
# Only one job (previous .cache() did not trigger a job)

52455

In [4]:
rdd.take(3)
# [(line_number, partition)]

[(0, ''),
 (8,
  '\r\nWARC-Type: warcinfo\r\nWARC-Date: 2020-04-10T16:44:20Z\r\nWARC-Filename: CC-MAIN-20200328074047-20200328104047-00000.warc.wet.gz\r\nWARC-Record-ID: <urn:uuid:3b06c614-57de-423a-aba2-2e8c39f02c0b>\r\nContent-Type: application/warc-fields\r\nContent-Length: 376\r\n\r\nSoftware-Info: ia-web-commons.1.1.10-SNAPSHOT-20200324091056\r\nExtracted-Date: Fri, 10 Apr 2020 16:44:20 GMT\r\nrobots: checked via crawler-commons 1.1-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)\r\nisPartOf: CC-MAIN-2020-16\r\noperator: Common Crawl Admin (info@commoncrawl.org)\r\ndescription: Wide crawl of the web for March/April 2020\r\npublisher: Common Crawl\r\n\r\n\r\n\r\n'),
 (651,
  '\r\nWARC-Type: conversion\r\nWARC-Target-URI: http://000ojfb.wcomhost.com/ushwa/2018-dan-patch-banquet/ken-weingartner-chof/\r\nWARC-Date: 2020-03-28T09:27:14Z\r\nWARC-Record-ID: <urn:uuid:1ebf934c-633e-4362-ac23-b2b17993ec64>\r\nWARC-Refers-To: <urn:uuid:7ee10237-5635-4404-9a2c-b546e8aef370>\r\n

In [5]:
rdd.getNumPartitions()

4

In [6]:
print(spark_context.uiWebUrl)

http://host-192-168-2-218-de1:4040


In [7]:
## Example #1 - Filter by Top_level Domain and compute most common words ##

# Try .ac.uk, .ru, .se, .com
p = re.compile('WARC-Target-URI: \S+\.ac.uk', re.IGNORECASE)


# Note: .partition(..) returns a 3-tuple: the string before the separator (index 0), 
# the separotor (index 1), and the part of the string afterwards (index 2) -- which is the part we want.
all_words = rdd\
    .filter(lambda doc: bool(p.search(doc[1])))\
    .map(lambda web_text: web_text[1].partition('\r\n\r\n')[2])\
    .flatMap(lambda t: t.split(' '))\
    .flatMap(lambda w: w.split('\n'))\



all_words_and_count = all_words.map(lambda w: w.strip())\
    .map(lambda w: (w,1))


word_counts = all_words_and_count.reduceByKey(add)

print(word_counts.takeOrdered(60, key=lambda x: -x[1]))



[('and', 2044), ('the', 1962), ('of', 1597), ('to', 1166), ('1', 994), ('in', 786), ('a', 717), ('for', 552), ('-', 521), ('The', 373), ('on', 339), ('&', 328), ('is', 324), ('Research', 306), ('Photographic', 287), ('collection:', 286), ('at', 278), ('with', 278), ('by', 272), ('', 261), ('University', 261), ('(←', 250), ('links)', 250), ('\u200e', 250), ('|', 235), ('that', 232), ('College', 229), ('view', 227), ('this', 225), ('as', 218), ('webpages', 211), ('are', 203), ('from', 175), ('be', 170), ('About', 168), ('hircus', 168), ('you', 162), ('Contact', 161), ('your', 161), ('or', 158), ('Search', 153), ('2020', 148), ('our', 142), ('research', 141), ('2', 138), ('us', 136), ('Student', 125), ('was', 121), ('…', 120), ('Ulster', 120), ('News', 119), ('Events', 118), ('Information', 118), ('not', 118), ('have', 117), ('Centre', 115), ('information', 112), ('an', 110), ('more', 109), ('results', 105)]


In [8]:
## Example #2 - Group by TLD and compute most common words for each ##

ex = "WARC-Type: conversion\
WARC-Target-URI: http://news.bbc.co.uk/2/hi/africa/3414345.stm\
WARC-Date: 2014-08-02T09:52:13Z"

p = re.compile('WARC-Target-URI: \S+\.([a-zA-Z]{2,3})/', re.IGNORECASE)
# print(p.search(ex).group(1))
# uk

def get_tld(content):
    match = p.search(content)
    if match is not None:
        return match.group(1)
    else:
        return None

# discard the line number
# partition() -- python function -- split on the first occurance, returns (before,split,after)
# filter out those with no TLD

    
words_by_tld_rdd = rdd\
    .map(lambda filename_content: filename_content[1])\
    .map(lambda content: (get_tld(content), content.partition('\r\n\r\n')[2]))\
    .filter(lambda tld_content: tld_content[0] is not None)\
    .flatMapValues(lambda words: words.split(' '))\
    .flatMapValues(lambda words: words.split('\n'))\
    .mapValues(lambda word: word.strip())
    #.take(10)

# print(words_by_tld_rdd.take(10))

tlds = words_by_tld_rdd.countByKey()
#print(tlds)

tlds = OrderedDict(sorted(tlds.items(), key = itemgetter(1), reverse = True))
# print(tlds)  

top_tlds = dict(itertools.islice(tlds.items(), 10))

# print(top_tlds)

print("Results:")

for tld in top_tlds:
    print(tld)
    top_words_for_tld = words_by_tld_rdd\
        .filter(lambda tld_word: tld_word[0] == tld)\
        .values()\
        .map(lambda w: (w,1))\
        .reduceByKey(add)\
        .takeOrdered(20, key=lambda x: -x[1])
    print(top_words_for_tld)

Results:
com


[('the', 353000), ('to', 279593), ('and', 265457), ('of', 222848), ('a', 221433), ('►', 209163), ('-', 184758), ('de', 174580), ('in', 170178), ('(1)', 164072), ('for', 115777), ('is', 104315), ('', 97845), ('I', 93513), ('&', 93378), ('on', 85918), ('you', 82373), ('|', 77107), ('with', 73787), ('The', 73512)]
org


[('and', 54841), ('the', 52193), ('of', 39935), ('to', 36461), ('', 29470), ('in', 26439), ('a', 25804), ('for', 18046), ('-', 15382), ('de', 14208), ('is', 13663), ('filter', 11608), ('Apply', 11319), ('on', 10898), ('The', 10564), ('that', 9797), ('|', 8886), ('(1)', 8715), ('by', 8597), ('&', 8544)]
ru


[('и', 58881), ('', 52163), ('в', 48109), ('на', 28793), ('для', 27479), ('с', 20867), ('-', 18422), ('по', 13915), ('не', 11568), ('В', 9391), ('от', 8600), ('к', 6905), ('из', 6779), ('о', 6449), ('—', 6379), ('что', 6370), ('1', 6286), ('за', 5760), ('–', 5550), ('или', 5014)]
de


[('und', 32347), ('der', 22602), ('die', 18979), ('in', 18978), ('-', 18282), ('für', 12652), ('von', 11731), ('&', 11287), ('Sie', 9889), ('mit', 9824), ('zu', 8952), ('den', 8919), ('', 8805), ('the', 7483), ('auf', 7206), ('im', 6983), ('and', 6489), ('ist', 6202), ('€', 6158), ('of', 6043)]
net


...

[('the', 27991), ('and', 24643), ('to', 22214), ('of', 18116), ('a', 13310), ('-', 13116), ('in', 12656), ('&', 10023), ('for', 10023), ('is', 6836), ('you', 6558), ('on', 6514), ('with', 6388), ('your', 5778), ('The', 5702), ('are', 4550), ('at', 4431), ('by', 4405), ('|', 4339), ('I', 4110)]
fr


[('de', 38094), ('à', 16696), ('et', 15222), ('la', 14193), ('le', 10635), (':', 10030), ('des', 9866), ('-', 9575), ('les', 8528), ('du', 8165), ('en', 8121), ('resultats)', 6564), ('pour', 6548), ('a', 5152), ('un', 4913), ('sur', 4614), ('', 4477), ('au', 3852), ('Répondre', 3846), ('dans', 3662)]
pl


[('-', 18694), ('i', 13861), ('w', 13429), ('z', 9398), ('na', 8628), ('do', 8617), ('się', 4890), ('o', 3827), ('od', 3662), ('nie', 3593), ('to', 3392), ('–', 3213), ('jest', 2739), ('zł', 2664), ('dla', 2512), ('', 2313), ('W', 2313), ('|', 2253), ('(W)', 1878), ('a', 1869)]
it


[('di', 19008), ('e', 14751), ('per', 7341), ('in', 7155), ('-', 6980), ('il', 6285), ('la', 6117), ('a', 6072), ('del', 5455), ('che', 3756), ('con', 3688), ('da', 3662), ('un', 3519), ('i', 3310), ('è', 3170), ('della', 3161), ('al', 3042), ('le', 2963), ('–', 2811), ('dei', 2796)]
edu


[('and', 10253), ('of', 10031), ('-', 10007), ('the', 9720), ('to', 7632), ('Week', 6874), ('in', 4653), ('a', 4360), ('for', 3477), ('at', 2918), ('1', 2850), ('J.', 2744), ('&', 2520), ('M.', 2372), ('I', 2224), ('2', 2055), ('on', 2052), ('A.', 1918), ('The', 1897), ('is', 1868)]


In [8]:
spark_session.stop()